# **4.2 범주형 데이터 다루기**

4.1절에서는 수치형 데이터만 사용했지만, **실제 데이터 셋에는 범주형 데이터도 포함**되어 있다.<br>

범주형 데이터는 순서가 있는 것과 없는 것을 구분해야 한다.
- **순서가 있는 특성**: 정렬하거나 차례대로 놓을 수 있다.
      ex) '최상 > 상 > 중 > 하 > 최하'
- **순서가 없는 특성**: 차례를 부여할 수 없다.
      ex) 축구 팀 이름: 맨체스터 유나이티드, 첼시, 리버풀, 아스날
---

#### 예제를 위한 범주형 데이터가 포함된 DataFrame

새로운 DataFrame에는 다양한 특성을 가진 데이터가 준비되어 있다.<br>
- **순서가 없는 특성**: ```color```
- **순서가 있는 특성**: ```size```
- **수치형 특성**: ```price```
- **클래스 레이블**: ```classlabel```

In [1]:
import pandas as pd
df = pd.DataFrame([
                   ['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


---

## **4.2.1 순서가 있는 특성 매핑**

학습 알고리즘이 순서가 있는 특성을 제대로 인식하려면 범주형 문자열 값을 정수형으로 바꿔야 한다.<br>

                                          XL = L + 1 = M + 2

In [2]:
size_mapping = {
                'XL': 3,
                'L': 2,
                'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


---

만약 정수 값을 **다시 원래 문자열로 바꾸고 싶다면** 거꾸로 매핑하는 딕셔너리를 정의하면 된다.<br><br>

```inv_size_mapping = {v: k for k, v in size_mapping.items()}```

<br>

```size_mapping``` 딕셔너리와 비슷하게 **판다스의 map 메소드를 사용**하여 변환된 특성 열에 적용할 수 있다.

In [3]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

---

수치적 **크기에 대해 확신이 없**거나 두 범주 사이의 **순서를 정의할 수 없다**면 임계 값을 사용하여 **0/1로 인코딩**할 수 있다.<br>

      - 임계값: 모델이 분류의 답을 결정할 때 기준 값


ex) ```M, L, XL 값을 가진 특성 size를 두 개의 새로운 특성 'x > M'과 'x > L'로 나눌 수 있다.```

In [20]:
import pandas as pd
test_df = pd.DataFrame([
                   ['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])
test_df.columns = ['color', 'size', 'price', 'classlabel']
test_df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


판다스 데이터프레임의 ```apply``` 메소드를 사용해서 **임계 값 기준으로 특성을 인코딩**하는 ```lambda``` 함수를 적용할 수 있다.

In [21]:
test_df['x > M'] = test_df['size'].apply(lambda x: 1 if x in {'L', 'XL'} else 0)
test_df['x > L'] = test_df['size'].apply(lambda x: 1 if x == 'XL' else 0)
del test_df['size']
test_df

,color,price,classlabel,x > M,x > L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1


## **4.2.2 클래스 레이블 인코딩**

사이킷런은 대부분 클래스 레이블을 정수로 변환해 주지만, 실수를 방지하기 위해서 **클래스 레이블을 정수 배열로 전달하는 것이 좋다**.<br>

클래스 레이블은 순서가 없기 때문에 특정 클래스 레이블에 할당한 **숫자는 의미가 없다.**

1. ```enumerate```를 사용하여 클래스 레이블을 0부터 할당한다.<br>
(```enumerate```는 반복 가능한 객체(문자열, 리스트, 넘파이 배열 등)를 입력으로 받아 인덱스와 값의 튜플을 차례대로 반환하는 파이썬 내장 함수이다.)

In [4]:
import numpy as np
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

2. 매핑 딕셔너리를 사용하여 클래스 레이블을 정수로 변환한다.

In [5]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


---

####**원본 문자열로 바꾸는 방법**

1. 다음과 같이 **매핑 딕셔너리의 키-값 쌍을 뒤집**는다.

In [6]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


2. 사이킷런에 구현된 **```LabelEncoder```를 사용**한다.<br>
(```LabelEncoder``` 객체의 ```classes_```속성에 각 클래스의 레이블이 저장되어 있다.)

In [7]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

    - fit_transform 메소드는 fit 메소드와 transform 메소드를 합쳐 놓은 단축 메소드이다.

3. **```inverse_transform``` 메소드를 사용**한다.

In [8]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

---

## **4.2.3 순서가 없는 특성에 원-핫 인코딩 적용**

순서가 없는 ```color``` 열에도 ```LabelEncoder```를 사용하여 문자열 레이블을 정수로 바꿀 수 있다.<br>



- blue = 0
- green = 1
- red = 2


In [9]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)


    이 코드에서 color 열만 추출(X[:, 0])해서 주입한 이유는 LabelEncoder는 입력 데이터로 1차원 배열을 받기 때문이다.
    1차원 배열을 받는 이유는 타깃 레이블을 인코딩하기 위해 만들어진 클래스이기 때문이다.

<br>

이 상태로 배열을 분류기에 넣으면 **문제가 발생**할 수 있다.<br>

```color```값에는 순서가 없지만, 알고리즘이 ```blue < green < red``` 라고 가정하고 학습을 할 수 있다.<br><br>

이 문제를 해결하기 위한 방법이 **원-핫 인코딩(one-hot encoding) 기법**이다.

### **One-hot encoding**

**One-hot encoding**은 순서 없는 특성에 들어 있는 **고유한 값마다 새로운 더미(dummy) 특성**을 만드는 것이다.<br><br>

1. ```color``` 특성을 세 개의 새로운 특성인 ```blue, green, red```로 변환한다.
2. 이진 값을 사용하여 특정 샘플의 ```color```를 나타낸다.
        blue 샘플: blue = 1, green = 0, red = 0
        green 샘플: blue = 0, green = 1, red = 0
        red 샘플: blue = 0, green = 0, red = 1


사이킷런의 ```preprocessing``` 모듈에 구현된 **```OneHotEncoder```를 사용**하면 된다.<br>

  - ```color``` 배열만 수정하기 위해 ```OneHotEncoder```를 하나의 열(```X[:, 0].reshape(-1, 1))```에만 적용했다.

In [10]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

여러 개의 특성이 있는 배열에서 **특정 열만 변환**하려면 **```ColumnTransformer```를 사용**한다.<br>
```ColumnTransformer```는 ```(name, transformer, column(s))``` 튜플의 리스트를 받는다.<br>

(*※ ColumnTransformer에 대한 추가적인 설명은 아래에 OrdinalEncoder와 함께 정리해두었다.*)

In [13]:
from sklearn.compose import ColumnTransformer
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
                              ('onehot', OneHotEncoder(), [0]),
                              ('nothing', 'passthrough', [1, 2])    # 'passthrough'를 사용하면 열을 변경하지 않는다.
])
c_transf.fit_transform(X)
ColumnTransformer()

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

원-핫 인코딩으로 **더미 변수를 만들 때**, 판다스의 **```get_dummies``` 메소드**를 사용하면 편리하다.<br>

**DataFrame에 적용하면** ```get_dummies``` 메소드는 **문자열 열만 변환**하고 나머지 열은 그대로 놔둔다.

In [14]:
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


```get_dummies``` 메소드에서 **```columns``` 매개변수**를 사용하면 **변환하려는 특성을 구체적으로 지정**할 수 있다.

In [19]:
pd.get_dummies(df[['price', 'color', 'size']], columns=['size'])

,price,color,size_1,size_2,size_3
0,10.1,green,1,0,0
1,13.5,red,0,1,0
2,15.3,blue,0,0,1


---

<br>

**다중 공선성(multicollinearity)**<br>
원-핫 인코딩된 데이터셋을 사용할 땐 다중 공선성 문제를 유념해야 한다.<br>

역행렬을 구해야 하는 경우에는 문제가 된다.

    다중 공선성: 통계학의 회귀분석에서 독립변수들 간에 강한 상관관계가 나타나는 문제

특성 간 **상관관계가 높으면 역행렬을 계산하기 어려워** 수치적으로 불안해진다.<br>

상관관계를 줄이려면 원-핫 인코딩된 배열에서 **특성 열 하나를 제거**하면 된다.<br> 

이렇게 특성을 제거하더라도 **잃는 정보는 없다.**<br><br>

ex)  ```color_green = 0이고, color_red = 0일 때, blue임을 알 수 있다.```

<br><br>
```get_dummies``` 메소드의 **```drop_first``` 매개변수를 ```True```**로 지정하면 **첫 번째 열을 삭제**할 수 있다.

In [16]:
pd.get_dummies(df[['price', 'color', 'size']],
              drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


```OneHotEncoder```에서 **중복된 열을 삭제**하려면 **```drop='first'```와 ```categories='auto'```**로 지정하면 된다.

In [17]:
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([
                              ('onehot', color_ohe, [0]),
                              ('nothing', 'passthrough', [1, 2])
])
c_transf.fit_transform(X)

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)

### **OrdinalEncoder & ColumnTransformer**

```OrdinalEncoder```와 ```ColumnTransformer```를 함께 사용하면 **여러 개의 열을 한 번에 정수로 변환**할 수 있다.
- **```OrdinalEncoder```**: 범주형 데이터를 정수로 인코딩하는 클래스

- **```ColumnTransformer```**: pandas 데이터프레임의 열마다 다른 변환을 적용하도록 도와주는 클래스

#### **```ColumnTransformer``` 클래스**

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(dtype=np.int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['color'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

```ColumnTransformer```는 첫 번째 매개변수로 트랜스포머(transformer)의 리스트를 받는다.<br>

    def __init__(transformers, remainder='drop', sparse_threshold=0.3, n_jobs=None, transformer_weights=None, verbose=False)

**트랜스포머(transformer)**: 이름, 변환기, 변환할 열의 리스트로 이루어진 튜플
- 이름: ```ord_enc```<br>

- 변환기: ```OrdinalEncoder``` 객체<br>

- 변환할 열: ```color```

<br>

위 코드의 결과는 ```color``` 열이 정수 값으로 변환되었다.<br><br>


```ColumnTransformer```에 사용한 변환기는 ```named_transformers_``` 속성에서 앞서 지정한 ord_enc 이름으로 참조할 수 있다.<br>

정수로 인코딩된 값을 **다시 문자열로 변환**하려면 ```inverse_transform``` 메소드를 호출하면 된다.

In [ ]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

#### **```OrdinalEncoder``` 클래스**

    def __init__(categories='auto', dtype=np.float64)
**```dtype```** 매개변수:
- 기본값: **```np.float64```**, 실수로 인코딩한다.
- **정수**로 인코딩하고 싶으면 **```np.int```를 지정**하면 된다.

<br>

**```categories```** 매개변수:
- 기본값: ```auto```, 훈련 데이터셋에서 **자동으로 범주를 인식**한다.
- ```categories``` 매개변수에 **직접 범주 리스트를 전달**할 수 있다.
- 인식된 범주는 **```categories_``` 속성에 저장**된다.
